In [ ]:
import pandas as pd
from dateutil.relativedelta import relativedelta

# CHANGE THESE TWO VALUES
input_file_path = 'Nifty50.csv' #Must be csv
output_file_path = 'Nifty50Output.xlsx' #xlsx

return_columns = ['12M_Simple_Return', '12M_Complex_Return']

def calculate_percentages(df):
    # Calculate percentages for each column
    total_positive_percent = 0
    total_negative_percent = 0
    
    for column in return_columns:
        total_count = df[column].notna().sum()
        positive_count = (df[column] > 0).sum()
        negative_count = (df[column] < 0).sum()
        
        positive_percent = (positive_count / total_count) * 100
        negative_percent = (negative_count / total_count) * 100
        
        total_positive_percent += positive_percent
        total_negative_percent += negative_percent
    
    num_columns = len(return_columns)
    avg_positive = total_positive_percent / num_columns
    avg_negative = total_negative_percent / num_columns
    
    print("\nTime spent in:")
    print(f"Positive %: {avg_positive:.1f}%")
    print(f"Negative %: {avg_negative:.1f}%")


# Read the Excel file
df = pd.read_csv(input_file_path)
# Ignoring extra columns
df = df[['Date', 'Close']]
# Convert date column to datetime
try:
    df['Date'] = pd.to_datetime(df['Date'], format='%d-%b-%y')
except:
    df['Date'] = pd.to_datetime(df['Date'])
# Sort values by date
df = df.sort_values('Date').reset_index(drop=True)


df['12M_Simple_Return'] = None
df['12M_Complex_Return'] = None

# ~~~~~~~~~~~~~~~ CALCULATING THROUGH SIMPLE METHOD ~~~~~~~~~~~~~~~

# Calculate various rolling returns
# 12-month rolling return
df['12M_Simple_Return'] = df['Close'].pct_change(periods=365) * 100


# ~~~~~~~~~~~~~~~ CALCULATING THROUGH COMPLEX METHOD ~~~~~~~~~~~~~~~

# Function to calculate returns between two dates
def calculate_return(current_value, past_value):
    if pd.isna(past_value) or pd.isna(current_value):
        return None
    return ((current_value - past_value) / past_value) * 100

# Calculate returns for each row
for i in range(len(df)):
    current_date = df.loc[i, 'Date']
    current_value = df.loc[i, 'Close']
    
    # Calculate dates for different lookback periods
    twelve_months_ago = current_date - relativedelta(months=12)
    
    # Find closest previous values for each period
    def get_closest_past_value(target_date):
        past_data = df[df['Date'] <= target_date]
        if len(past_data) > 0:
            return past_data.iloc[-1]['Close']
        return None
    
    # Calculate returns
    df.loc[i, '12M_Complex_Return'] = calculate_return(current_value, get_closest_past_value(twelve_months_ago))

# Comment out the below line if you don't need an output file
df.to_excel(output_file_path, index=False)

# Average value calculation
SimpleValue = df['12M_Simple_Return'].mean()  
ComplexValue = df['12M_Complex_Return'].mean()
finalValue = (SimpleValue + ComplexValue) / 2

print("\nSimple calculation, rolling return is {}".format(SimpleValue.round(2)))
print("Complex calculation, rolling return is {}".format(ComplexValue.round(2)))
print("Final rolling return is {}".format(finalValue.round(2)))
calculate_percentages(df)


2005-04-01 00:00:00
2020-02-28 00:00:00

Simple calculation, rolling return is 29.06
Complex calculation, rolling return is 19.48
Final rolling return is 24.27

Time spent in:
Positive %: 71.2%
Negative %: 28.8%
